In [ ]:
!pip install normflows

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2, kstest
import normflows as nf
import json

In [ ]:
filename="/content/Zmumu.json"
with open(filename, 'r') as file:
    data = json.load(file)

In [ ]:
data2=np.array(data)

In [ ]:
data2=np.array(data)
mean = data2.mean(axis=0)
std = data2.std(axis=0)
data_std = (data2 - mean) / std

X = torch.tensor(data_std, dtype=torch.float32)

In [ ]:
dim = X.shape[1]
n_flows = 8
hidden_units = 512
num_bins = 12

# Base distribution
q0 = nf.distributions.base.DiagGaussian(dim)

# Flow layers
flows = []
for _ in range(n_flows):
    flows.append(nf.flows.AutoregressiveRationalQuadraticSpline(
        num_input_channels=dim,
        num_bins=num_bins,
        num_hidden_channels=hidden_units,
        num_blocks=1
    ))

In [ ]:
model = nf.NormalizingFlow(q0, flows)

In [ ]:
enable_cuda = True
device = torch.device('cuda' if torch.cuda.is_available() and enable_cuda else 'cpu')
model = model.to(device)

In [ ]:
from tqdm import tqdm
# Train model
max_iter = 3000
num_samples = 2 ** 9
show_iter = 10


#loss_hist = np.array([])

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for it in tqdm(range(max_iter)):
    optimizer.zero_grad()

    indices = torch.randperm(X.size(0))[:num_samples]

    # Select the sampled points using the generated indices
    x = X[indices].to(device)

    # Compute loss
    loss = model.forward_kld(x)

    # Do backprop and optimizer step
    if ~(torch.isnan(loss) | torch.isinf(loss)):
        loss.backward()
        optimizer.step()

    # Log loss
    loss_hist = np.append(loss_hist, loss.to('cpu').data.numpy())


# Plot loss
plt.figure(figsize=(10, 10))
plt.plot(loss_hist, label='loss')
plt.legend()
plt.show()

In [ ]:
with torch.no_grad():
    beta = model.inverse(X).numpy()
    norms_sq = np.sum(beta**2, axis=1)

In [ ]:
np.min(norms_sq)

In [ ]:
K = 11  # total bins
dof = 5
bin_edges = chi2.ppf(np.linspace(0, 1, K + 1), df=dof)

hist, _ = np.histogram(norms_sq, bins=bin_edges)
hist = hist[:10]  # ignore last bin
bin_edges = bin_edges[:11]  # only first 10 edges (10 bins)

bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])
bin_widths = np.diff(bin_edges)

# Normalized density
N = len(norms_sq)
hist_density = hist / (bin_widths * N)
errors = np.sqrt(hist) / (bin_widths * N)


expected_density = chi2.pdf(bin_centers, dof)
expected_counts = (chi2.cdf(bin_edges[1:], dof) - chi2.cdf(bin_edges[:-1], dof)) * N

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 6), gridspec_kw={'height_ratios': [3, 1]})


xerr = bin_widths / 2
ax1.errorbar(
    bin_centers,
    hist_density,
    yerr=errors,
    xerr=xerr,
    fmt='o',
    label='Transformed samples',
    capsize=3
)
x = np.linspace(0, 20, 1000)
ax1.plot(x, chi2.pdf(x, dof), 'r-', label=f"$\chi^2$(df={dof})")
ax1.set_yscale('log')
ax1.set_ylabel('Density')
ax1.set_title('Distribution of $||\\vec{\\beta}||^2$ (first 10 bins)')
ax1.legend()

residuals = hist - expected_counts
significance = residuals / np.sqrt(expected_counts)

ax2.bar(bin_centers, significance, width=bin_widths, color='gray', edgecolor='black')
for y in [-2, -1, 1, 2]:
    ax2.axhline(y, linestyle='--', color='red' if abs(y) == 2 else 'gold')
ax2.set_ylabel('Residuals (σ)')
ax2.set_xlabel(r"$\|\vec{\beta}\|^2$")
ax2.set_xlim(0, 20)

plt.tight_layout()
plt.show()